# Třídění stažených souborů - řešení

In [10]:
import pathlib
import shutil
import logging

logging.basicConfig(
    format='%(levelname)s - %(message)s',
    level=logging.DEBUG,
)


DRY_RUN = True
PATH = pathlib.Path("/path/to/downloads")

map_ = {
    "document": ["pdf", "epub", "djvu", "docx", "md"],
    "video": ["mp4", "webm", "mkv"],
    "python": ["py"],
    "sw": ["iso", "deb", "run"],
    "image": ["jpeg", "jpg", "png", "webp"],
}

output_dirs = {
    "document": "docs",
    "video": "videos",
    "image": "images",
    "sw": "software",
    "python": "python"
}


def invert_map(map_: dict) -> dict:
    """
    Inverts the 'dest: [exts]' map to 'ext: dir'
    """
    return {ext: output_dirs[cat] for cat, exts in map_.items() for ext in exts}


def prepare_dirs(output_dirs: dict) -> None:
    """
    Creates all dest. directories
    """
    logging.debug("creating directory tree")
    
    for directory in output_dirs.values():
        logging.debug(f"creating directory '{directory}'")
        if not DRY_RUN:
            (PATH / directory).mkdir(exist_ok=True)


def main():
    logging.info(f"starting sorting for {PATH}")
    
    if DRY_RUN:
        logging.info(f"dry run is on, nothing will be moved")
        
    prepare_dirs(output_dirs)
    outmap = invert_map(map_)
    
    files = [file for file in PATH.iterdir() if not file.is_dir()]
    
    for file in files:
        ext = file.suffix[1:]
        
        try:
            where_to = outmap[ext]
        except KeyError as e:
            logging.info(f"skipping '{file.name}' - do not know what to do with extension '{ext}'")
            continue
            
        dst = (PATH / where_to) / file.name
        
        logging.info(f"moving '{file.name}' to '{dst.parent.stem}'")
        if not DRY_RUN:
            shutil.move(file, dst)
            